# RAG 실습: Amazon Bedrock을 이용한 chat assistant

이 노트북은 Amazon Bedrock에 호스팅된 대형 언어 모델(LLM)을 사용하여 챗봇을 만드는 과정을 안내합니다. 또한, 지식 베이스를 활용해 데이터를 벡터화하고 저장하며, 의미 기반 검색을 통해 데이터를 검색하는 방법도 다룹니다. 벡터 인덱스에는 Amazon OpenSearch Serverless를 사용할 것입니다.

우리는 LLM과 지식 베이스(KB)와의 연동을 쉽게 하기 위해서 LangChain을 사용할 것입니다. 이 노트북을 진행하면서 Amazon Bedrock 클라이언트 환경 설정, 보안 권한 구성, 그리고 LangChain에서 프롬프트 템플릿을 사용하는 방법을 배우게 됩니다.

<div class="alert alert-block alert-info">
<b>Note:</b>
    <ul>
        <li>이 노트북은 Amazon SageMaker 노트북 <a href="https://docs.aws.amazon.com/sagemaker/latest/dg/nbi.html">인스턴스 또는 Amazon SageMaker 스튜디오 노트북</a> <a href="https://docs.aws.amazon.com/sagemaker/latest/dg/notebooks.html">및 Amazon OpenSearch</a> 서버리스를 지원하는 AWS 지역에서만 실행해야</a> 합니다. <a href="https://aws.amazon.com/opensearch-service/features/serverless/">.</li>
        <li>이 노트북을 작성할 당시에는 Amazon Bedrock을 <a href="https://docs.aws.amazon.com/bedrock/latest/userguide/what-is-bedrock.html#bedrock-regions">지원되는 AWS</a> 지역에서만 사용할 수 있었습니다. 다른 AWS 지역에서 이 노트북을 실행하는 경우 Amazon Bedrock 클라이언트의 지역 및/또는 엔드포인트 URL 파라미터를 지원되는 AWS 지역 중 하나로 변경해야 합니다. 이 노트북의 <i>가져오기 구성</i> 섹션에 있는 지침을 따르십시오.</li>
        <li>이 노트북은 <i>ml.m5.xlarge의</i> 최소 인스턴스 크기로 실행하는 것이 좋습니다.
             <ul>
             <li><i>아마존 리눅스 2, 주피터 랩 3을</i> 아마존 세이지메이커 노트북 인스턴스의 플랫폼 식별자로 사용하는 경우</li>
             <li>(또는)
             <li><i>데이터 사이언스 3.0을</i> 아마존 세이지메이커 스튜디오 노트북의 이미지로 사용합니다.</li>
        </li><ul>
             <li>이 글을 쓰는 시점에서 이 노트북을 실행하는 데 가장 적합한 최신 버전의 커널은
             <ul>
             <li><i>Amazon SageMaker Notebook 인스턴스에는 conda_python3이 있습니다.</i></li>
             <li><i>Amazon SageMaker Studio Notebook에는 python3이 있습니다.</i></li>
            </ul>
        </li>
    </ul>
</div>

## 1.	사전 준비

사전 준비를 확인하고 완료하세요.

### A. 인터넷 접근 확인 및 구성

이 노트북은 필요한 소프트웨어 업데이트를 다운로드하고 데이터셋을 가져오기 위해 인터넷에 대한 아웃바운드 접근이 필요합니다. 기본 설정으로 직접 인터넷 접근을 제공하거나, Amazon VPC를 통해 인터넷 접근을 제공할 수 있습니다.

### B. 필요한 소프트웨어 라이브러리 설치

이 노트북에는 다음 라이브러리들이 필요합니다:

- [SageMaker Python SDK 버전 2.x](https://sagemaker.readthedocs.io/en/stable/v2.html)
- [Python 3.10.x](https://www.python.org/downloads/release/python-3100/)
- [Boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html)
- [LangChain](https://www.langchain.com/)

다음 셀을 실행하여 필요한 라이브러리들을 설치하세요.

In [ ]:
!pip install boto3==1.34.51
!pip install botocore==1.34.142
!pip install langchain==0.0.339
!pip install unstructured==0.11.0
!pip install opensearch-py==2.4.2
!pip install PyPDF2==3.0.1
!pip install requests-aws4auth==1.2.3

import IPython

IPython.Application.instance().kernel.do_shutdown(True)

### C. 로깅 설정

**a. 시스템 로그 (선택 사항)**

시스템 로그는 노트북이 기본 노트북 인스턴스와 상호작용하면서 생성되는 로그를 의미합니다. 예를 들어, 노트북을 로드하거나 저장할 때 생성되는 로그가 이에 해당합니다.

이 로그는 노트북 인스턴스가 시작될 때 자동으로 설정됩니다.

이 로그는 이 노트북이 실행 중인 동일한 AWS 리전의 Amazon CloudWatch Logs 콘솔을 통해 액세스할 수 있습니다.

- 이 노트북을 Amazon SageMaker Notebook 인스턴스에서 실행 중인 경우, 다음 위치로 이동합니다:
  - CloudWatch > Log groups > /aws/sagemaker/NotebookInstances > {notebook-instance-name}/jupyter.log
- 이 노트북을 Amazon SageMaker Studio Notebook에서 실행 중인 경우, 다음 위치로 이동합니다:
  - CloudWatch > Log groups > /aws/sagemaker/studio > {sagmaker-domain-name}/{user-name}/KernelGateway/{notebook-instance-name}
  - CloudWatch > Log groups > /aws/sagemaker/studio > {sagmaker-domain-name}/{user-name}/JupyterServer/default

이 노트북이 실행 중인 기본 인스턴스의 이름을 확인하려면, 아래의 코드 셀의 주석을 해제하고 실행하세요.

In [ ]:
import json

notebook_name = ''
resource_metadata_path = '/opt/ml/metadata/resource-metadata.json'
with open(resource_metadata_path, 'r') as metadata:
    notebook_name = (json.load(metadata))['ResourceName']
print("Notebook instance name: '{}'".format(notebook_name))

**b. 애플리케이션 로그**

애플리케이션 로그는 이 노트북에서 다양한 코드 셀을 실행할 때 생성되는 로그를 의미합니다. 이를 설정하려면, 아래의 셀을 실행하여 Python 로깅 서비스를 인스턴스화하세요. 필요한 경우 기본 로그 수준과 형식을 구성할 수 있습니다.

기본적으로, 이 노트북은 로그를 해당 셀의 출력 콘솔에만 표시합니다.

In [ ]:
import logging
import os

# Set the logging level and format
log_level = logging.INFO
log_format = '%(asctime)s - %(levelname)s - %(message)s'
logging.basicConfig(level=log_level, format=log_format)

# Save these in the environment variables for use in the helper scripts
os.environ['LOG_LEVEL'] = str(log_level)
os.environ['LOG_FORMAT'] = log_format

### D. import 준비
나중에 사용할 수 있도록 모든 라이브러리 및 모듈 import를 준비합니다.

In [ ]:
import boto3
import langchain
import opensearchpy
import PyPDF2
import requests
import sagemaker
import sys
from botocore.config import Config
from tqdm.contrib.concurrent import process_map
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.embeddings import BedrockEmbeddings
from langchain.llms import Bedrock
from langchain.memory import ConversationBufferMemory
from langchain.prompts.prompt import PromptTemplate
from langchain.vectorstores import OpenSearchVectorSearch
from IPython.core.display import HTML

# Import the helper functions from the 'scripts' folder
sys.path.append(os.path.join(os.getcwd(), "scripts"))
#logging.info("Updated sys.path: {}".format(sys.path))
from helper_functions import *

In [ ]:
logging.info("Python version : {}".format(sys.version))
logging.info("Boto3 version : {}".format(boto3.__version__))
logging.info("SageMaker Python SDK version : {}".format(sagemaker.__version__))
logging.info("LangChain version : {}".format(langchain.__version__))
logging.info("OpenSearch Python Client version : {}".format(opensearchpy.__version__))
logging.info("PyPDF2 version : {}".format(PyPDF2.__version__))

### E. AWS 리전 및 boto3 구성 설정
현재 AWS 리전(이 노트북이 실행 중인 곳)과 SageMaker 세션을 가져옵니다. 이는 boto3 API를 사용해 일부 클라이언트를 AWS 서비스로 초기화하는 데 사용됩니다. Bedrock 기본 리전으로는 Oregon(us-west-2) 리전을 추천합니다.

In [ ]:
# Get the AWS Region, SageMaker Session and IAM Role references
my_session = boto3.session.Session()
logging.info("SageMaker Session: {}".format(my_session))
my_iam_role = sagemaker.get_execution_role()
logging.info("Notebook IAM Role: {}".format(my_iam_role))
my_region = my_session.region_name
logging.info("Current AWS Region: {}".format(my_region))

# Explicity set the AWS Region for Amazon Bedrock clients
AMAZON_BEDROCK_DEFAULT_REGION = "us-west-2"
br_region = os.environ.get('AMAZON_BEDROCK_REGION')
if br_region is None:
    br_region = AMAZON_BEDROCK_DEFAULT_REGION
elif len(br_region) == 0:
    br_region = AMAZON_BEDROCK_DEFAULT_REGION
logging.info("AWS Region for Amazon Bedrock: {}".format(br_region))

이 노트북에서 사용되는 모든 boto3 클라이언트에 적용될 시간 초과 및 재시도 구성을 설정합니다.

In [ ]:
# boto3 클라이언트의 표준 시간 초과 제한을 1분에서 3분으로 늘립니다. 그리고 재시도 제한을 설정합니다.
my_boto3_config = Config(
    connect_timeout = (60 * 3),
    read_timeout = (60 * 3),
    retries = {
        'max_attempts': 10,
        'mode': 'standard'
    }
)

### F. Amazon OpenSearch Serverless collection 확인 및 생성

이 노트북은 벡터 검색 유형의 Amazon OpenSearch Serverless (AOSS) 컬렉션을 채팅 도우미가 사용할 벡터 데이터베이스로 사용합니다.

다음 셀을 실행하여 AOSS 컬렉션이 존재하지 않는지 확인하고 생성하십시오.


In [1]:
# AOSS 컬렉션이 존재하는지, 그리고 이 노트북을 통해 생성되었는지를 식별하기 위한 플래그를 설정합니다.
aoss_collection_exists = False
aoss_collection_created = False

> **Note:** 
이 노트북을 실행하려면 빈 컬렉션을 사용하는 것이 좋습니다. 세션 중에는 기본적으로 Vector search 유형의 빈 컬렉션이 미리 생성되어 바로 사용할 수 있습니다. 

다음 코드 셀을 실행하여 사용 가능한 첫 번째 AOSS 컬렉션의 세부 정보를 검색합니다.

In [ ]:
# Create the AOSS client
aoss_client = boto3.client("opensearchserverless", config = my_boto3_config)

# Check and create a collection if none is found
collection_id = ''
collections = aoss_client.list_collections()['collectionSummaries']
if len(collections) == 0:
    aoss_collection_exists = False
    logging.info("No AOSS collections exist.")
else:
    aoss_collection_exists = True
    logging.info("Found an AOSS collection.")
    first_collection = collections[0]
    collection_id = first_collection["id"]
    collection_name = first_collection["name"]

In [ ]:
if len(collection_id) == 0:
    aoss_collection_exists = False
    logging.info("No AOSS collections exist.")
else:
    aoss_collection_exists = True
    logging.info("AOSS 컬렉션은 Collectionid: {}; 컬렉션 이름: {} 을 사용합니다."
                 .format(collection_id, collection_name))
    # Print the AWS console URL to the AOSS collection
    collection_aws_console_url = "https://{}.console.aws.amazon.com/aos/home?region={}#opensearch/collections/{}"\
    .format(my_region, my_region, collection_name)
    logging.info("이 컬렉션을 보고 싶다면 {} 를 방문하십시오.".format(collection_aws_console_url))

### G. Amazon Bedrock에서 모델 액세스 활성화

> **Note:**
Amazon Bedrock에서 모델을 호출하기 전에 [여기](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html)의 지침에 따라 해당 모델에 대한 액세스를 활성화하십시오.


다음 셀을 실행하여 앞서 선택한 AWS 리전에 대한 Amazon Bedrock 모델 액세스 페이지 URL을 인쇄합니다.

In [ ]:
# 아마존 베드락 모델 액세스 페이지 URL을 인쇄합니다.
logging.info("Amazon Bedrock model access page - https://{}.console.aws.amazon.com/bedrock/home?region={}#/modelaccess"
             .format(br_region, br_region))

### H. 보안 권한 확인 및 구성

이 노트북은 기본 노트북 인스턴스에 연결된 IAM 역할을 사용합니다.이 역할의 이름을 보려면 다음 셀을 실행하십시오.

이 IAM 역할에는 다음과 같은 권한이 있어야 합니다.

 - Amazon Bedrock에서 대규모 언어 모델 (LLM) 을 호출할 수 있는 전체 액세스 권한.
 - 이전 단계에서 생성한 Amazon OpenSearch 서버리스 컬렉션에 대한 전체 읽기 및 쓰기 권한.
 - 아마존 CloudWatch 로그에 쓸 수 있는 액세스 권한.

또한 이 노트북 인스턴스와 연결된 IAM 역할에 대한 생성, 읽기 및 쓰기 액세스를 제공하려면 Amazon OpenSearch 서버리스 컬렉션에 데이터 액세스 제어를 설정해야 합니다.

In [ ]:
# IAM 역할 ARN 및 콘솔 URL을 인쇄합니다.
logging.info("This notebook's IAM role is '{}'".format(my_iam_role))
arn_parts = my_iam_role.split('/')
logging.info("Details of this IAM role are available at https://{}.console.aws.amazon.com/iamv2/home?region={}#/roles/details/{}?section=permissions"
             .format(my_region, my_region, arn_parts[len(arn_parts) - 1]))

### I. 공통 개체 만들기

먼저 다음 셀을 실행하여 Amazon Bedrock에서 사용 가능한 모든 모델을 나열하십시오. 그러면 이 노트북에서 사용할 Amazon Bedrock 내의 LLM 및 임베딩 모델을 선택하는 데 도움이 됩니다. 기본적으로 두 모델 모두 온디맨드 요금 모델을 사용합니다.

In [ ]:
# Amazon Bedrock에서 사용 가능한 모든 기반 모델을 나열하십시오.
models_info = ''
bedrock_client = boto3.client("bedrock", region_name = br_region, endpoint_url = "https://bedrock.{}.amazonaws.com"
                              .format(br_region), config = my_boto3_config)
response = bedrock_client.list_foundation_models()
model_summaries = response["modelSummaries"]
models_info = models_info + "\n"
models_info = models_info + "-".ljust(125, "-") + "\n"
models_info = models_info + "{:<15} {:<30} {:<20} {:<20} {:<40}".format("Provider Name", "Model Name", "Input Modalities",
                                                          "Output Modalities", "Model Id")
models_info = models_info + "-".ljust(125, "-")
for model_summary in model_summaries:
    models_info = models_info + "\n"
    models_info = models_info + "{:<15} {:<30} {:<20} {:<20} {:<40}".format(model_summary["providerName"],
                                                                            model_summary["modelName"],
                                                                            "|".join(model_summary["inputModalities"]),
                                                                            "|".join(model_summary["outputModalities"]),
                                                                            model_summary["modelId"])
models_info = models_info + "-".ljust(125, "-") + "\n"
logging.info("Displaying available models in the '{}' Region:".format(br_region) + models_info)

위 셀을 실행한 결과로부터

 1. 원하는 LLM에 해당하는 모델 ID를 선택하고 다음 셀의 llm_model_id 변수 값으로 설정합니다.
 2. (선택 사항) model_kwargs 매개변수에 [LLM별 추론 매개변수](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters.html)를 지정합니다.
 3. 원하는 임베딩 모델에 해당하는 모델 ID를 선택하고 다음 셀의 embeddings_model_id 변수 값으로 설정합니다.

이제 다음 셀을 실행하여 이 노트북의 이후 단계에서 사용할 공통 객체를 생성하십시오.

<div class="alert alert-block alert-info">
<b>Note:</b> 이 노트북은 다음 Amazon Bedrock 모델에서 테스트되었습니다:
    <li>LLMs: anthropic.claude-v2, anthropic.claude-instant-v1, cohere.command-text-v14, ai21.j2-ultra-v1</li>
    <li>Embedding model(s): amazon.titan-embed-text-v1</li>
</div>

<div class="alert alert-block alert-danger">
<b>Note:</b> 세션 중에는 다음과 같은 Amazon Bedrock 모델만 사용하는 것이 좋습니다:
    <li>LLMs: anthropic.claude-instant-v1</li>
    <li>Embedding model(s): amazon.titan-embed-text-v1</li>
</div>

In [ ]:
##### Specify the model-ids along with their inference parameters
# Model-id of the LLM to be used in the chat assistant
llm_model_id = "anthropic.claude-instant-v1"
temperature = 0.5
max_response_token_length = 300
# Model-id of the Embeddings model to be used in the chat assistant
embeddings_model_id = "amazon.titan-embed-text-v1"

##### LLM related objects
# Create the Amazon Bedrock runtime client
bedrock_rt_client = boto3.client("bedrock-runtime", region_name = br_region, config = my_boto3_config)

# Create the LangChain client for the LLM using the Bedrock client created above.
llm = Bedrock(
    model_id = llm_model_id,
    model_kwargs = get_model_specific_inference_params(llm_model_id,
                                                       temperature,
                                                       max_response_token_length),
    client = bedrock_rt_client
)

##### Embeddings related objects
# Use the LangChain BedrockEmbeddings class to create the Embeddings client.
br_embeddings = BedrockEmbeddings(client = bedrock_rt_client, model_id = embeddings_model_id, region_name = br_region)

##### Amazon OpenSearch Serverless (AOSS) related objects
# Create the AOSS Python client from the AOSS boto3 client using the helper function 
# available through ./scripts/helper_functions.py)
aoss_py_client = auth_opensearch(host = "{}.{}.aoss.amazonaws.com".format(collection_id, my_region),
                            service = 'aoss', region = my_region)
# Specify the name of the index in the AOSS collection; this will be created later in the notebook
index_name = "rag-with-gen-ai-index"
# Specify the max workers for loading data in parallel into the index
max_workers = 8
# To access an Opensearch Collection using LangChain, we can use the OpenSearchVectorSearch class.
doc_search = OpenSearchVectorSearch(
    opensearch_url = "{}.{}.aoss.amazonaws.com".format(collection_id, my_region),
    index_name = index_name,
    embedding_function = br_embeddings)
# Set the doc search client to the AOSS Python client
doc_search.client = aoss_py_client

##### File related objects
# Specify the path to the directory that will contain the RAG data
rag_dir = os.path.join(os.getcwd(), "data/rag")
# Create the directory if it doesn't exist
os.makedirs(rag_dir, exist_ok = True)

### J. Amazon OpenSearch Serverless collection에서 색인 생성

Amazon OpenSearch Serverless(AOSS) 컬렉션에서 인덱스를 생성하려면 먼저 인덱스의 스키마를 정의해야 합니다. AOSS를 사용하면 키워드 매칭을 활용하는 단순 검색 색인을 지정하거나 [k-최근접이웃 (k-NN) 검색](https://docs.aws.amazon.com/opensearch-service/latest/developerguide/knn.html)을 활용하는 벡터 검색 기능을 지정할 수 있습니다. 벡터 검색은 일반적인 키워드 매칭 또는 퍼지 매칭 알고리즘 대신 두 텍스트의 [임베딩](https://en.wikipedia.org/wiki/Word_embedding)을 비교한다는 점에서 표준 검색과 다릅니다. 임베딩은 텍스트와 같은 정보를 수치로 표현한 것으로, 이를 다른 임베딩과 비교할 수 있습니다. 임베딩에 대해 자세히 알아보려면 [이 블로그](https://huggingface.co/blog/getting-started-with-embeddings)를 참조하십시오. 벡터 검색 기능을 사용하면 최종 사용자가 채팅 도우미에게 보내는 질문과 의미상 유사한 문서를 검색할 수 있습니다. 이렇게 하면 사용자의 질문에 답하기 위해 LLM에 제공하는 컨텍스트를 개선할 수 있습니다.

In [ ]:
# k-NN 유형 벡터를 임베딩으로 사용하여 인덱스에 대한 스키마를 정의합니다.
knn_index = {
    "settings": {
        "index.knn": True,
    },
    "mappings": {
        "properties": {
           "content-embedding": { 
                "type": "knn_vector",
                "dimension": 1536 # can have dimension up to 10k
            },
            "content": {
                "type": "text"
            },
            "title": {
                "type": "text"
            },
            "source": {
                "type": "text"
            }
        }
    }
}

# Delete the index
#aoss_py_client.indices.delete(index = index_name)

# Create the index if it does not exist
if aoss_py_client.indices.exists(index = index_name):
    logging.info("AOSS index '{}' already exists.".format(index_name))
else:
    logging.info("Creating AOSS index '{}'...".format(index_name))
    logging.info(aoss_py_client.indices.create(index = index_name, body = knn_index, ignore = 400))

In [ ]:
# AWS 콘솔 URL을 AOSS 인덱스에 출력합니다.
index_aws_console_url = collection_aws_console_url + "/" + index_name
logging.info("이 색인을 보려면 {}를 방문하십시오.".format(index_aws_console_url))

## 2. 채팅 어시스턴트 빌드하기

대규모 언어 모델 (LLM)은 [환각](https://en.wikipedia.org/wiki/Hallucination_(artificial_intelligence)) 경향이 있습니다. LLM 컨텍스트에서의 환각은 확실하지만 사실적으로는 틀린 응답을 제공하는 우리의 모델입니다. 이 응답은 모델이 실제로 정답인지 여부에 관계없이 우리가 듣고 싶어한다고 생각하는 내용을 알려주는 경우가 많습니다. LLM이 잘못된 정보를 제공하지 못하도록 방지하는 한 가지 방법은 [검색 증강 생성 (RAG)](https://docs.aws.amazon.com/sagemaker/latest/dg/jumpstart-foundation-models-customize-rag.html) 메커니즘을 사용하는 것입니다.

RAG를 사용하면 모델이 학습 데이터로부터 사실을 기억하려고 애쓰는 대신 사실에 근거하여 응답하는 데 사용할 수 있는 올바른 컨텍스트 정보를 모델에 제공할 수 있습니다. RAG를 설정하려면 모델에 관련 소스 문서를 제공하는 데 활용할 수 있는 문서 데이터베이스가 있어야 합니다. 문서 데이터베이스를 설정하는 방법은 여러 가지가 있습니다. 이 노트북에서는 '벡터 검색' 유형의 [아마존 오픈서치 서버리스](https://docs.aws.amazon.com/opensearch-service/latest/developerguide/serverless-overview.html) 컬렉션을 사용할 것입니다.

[LangChain](https://www.langchain.com/)을 사용하여 채팅 어시스턴트가 수행하는 이벤트의 순서를 조정해 보겠습니다. LangChain은 LLM 애플리케이션 생성을 단순화하기 위해 설계된 프레임워크입니다. 유연한 추상화와 광범위한 툴킷을 제공합니다.

###  A. Architecture <a id='Architecture'></a>

![Architecture](./images/architecture.png)

### B. 0a단계: 벡터 데이터베이스에 데이터를 로드할 준비를 합니다.<a id='Step0a'></a>

Amazon OpenSearch Serverless (AOSS) 컬렉션은 특정 워크로드 또는 사용 사례를 지원하기 위해 함께 작동하는 하나 이상의 인덱스를 논리적으로 그룹화한 것입니다.

이 노트북은 벡터 인덱스를 사용하여 AOSS 컬렉션에 있는 문서를 인덱싱합니다. 처리할 수 있는 데이터의 다양성을 보여주기 위해 일부 HTML 및 PDF 파일에서 데이터를 수집해 보겠습니다. 수집 프로세스 중에 이러한 파일에서 텍스트를 추출하여 더 작은 청크로 분할합니다.

#### a. 텍스트 스플리터 초기화<a id='Initialize%20the%20text%20splitter'></a>

정보 검색을 위해 문서를 인덱싱할 때 전체 문서를 LLM에 컨텍스트로 제공하는 것은 LLM에 부담을 줄 수 있습니다. 특히 매우 긴 문서의 경우 더욱 그렇습니다. 가장 좋은 방법은 문서를 사용하기 쉬운 부분만 겹치는 청크로 나누는 것입니다. 찾고 있는 답변이 문서의 특정 구절 내에 포함되어 있고 전체 문서를 제공할 필요가 없는 경우가 많기 때문에 이러한 방식으로 문서를 나누면 검색 결과 관련성이 향상되는 경향이 있습니다. 

랭체인의 [RecursiveCharacterTextSplitter](https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter)를 사용하여 벡터 데이터베이스에 로드하기 전에 내용을 분할하는 데 사용할 텍스트 분할 객체를 만들어 보겠습니다. 여기서는 각 청크의 크기와 연속된 두 청크 사이의 겹치는 문자 수를 설정하는 간단한 '고정 크기 청킹' 전략을 사용하겠습니다.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 8000,
    chunk_overlap  = 100,
    length_function = len,
    add_start_index = True,
)

#### b. 로드할 HTML 파일을 준비합니다.<a id='Prepare%20HTML%20files%20for%20loading'></a>

채팅 어시스턴트가 Amazon Bedrock에 관한 질문에 사실적으로 정확한 정보로 답변할 수 있도록 다음 셀을 실행하여 Amazon Bedrock에 관한 일부 문서를 채우십시오.

이 설명서는 HTML 파일로 제공됩니다. 첫 단계로 이러한 파일을 `/data/rag/amazon-bedrock-docs/`라는 로컬 디렉터리에 다운로드합니다. 존재하지 않는 경우 필요한 디렉토리 구조가 생성됩니다.

In [ ]:
# A simple list of Agents for Amazon Bedrock documentation to index
html_link_list = [
    "https://docs.aws.amazon.com/bedrock/latest/userguide/what-is-service.html",
    "https://docs.aws.amazon.com/bedrock/latest/userguide/setting-up.html",
    "https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html",
    "https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters.html",
    "https://docs.aws.amazon.com/bedrock/latest/userguide/embeddings.html",
    "https://docs.aws.amazon.com/bedrock/latest/userguide/model-customization-prepare.html",
    "https://docs.aws.amazon.com/bedrock/latest/userguide/model-customization-guidelines.html",
    "https://docs.aws.amazon.com/bedrock/latest/userguide/prov-throughput.html",
    "https://docs.aws.amazon.com/bedrock/latest/userguide/agents.html",
    "https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base.html"
]

# Set the sub-directory to store these HTML files
html_files_dir = "{}/{}".format(rag_dir, "amazon-bedrock-docs")
# Create the directory if it doesn't exist
os.makedirs(html_files_dir, exist_ok = True)

# Download and store each HTML file; overwrite existing file
for html_link in html_link_list:
    html_content = requests.get(html_link).content.decode('utf-8')
    file_name = html_link.split("/")[-1]
    with open('{}/{}'.format(html_files_dir, file_name), "w") as f:
        f.write(html_content)
        logging.info("Downloaded file '{}' to '{}'".format(file_name, html_files_dir))

logging.info("A total of {} HTML files were downloaded to '{}'.".format(len(html_link_list), html_files_dir))
        
# Display the last downloaded HTML file
#HTML(html_content)

이제 HTML 문서를 모두 다운로드했으니 HTML 로더를 사용하여 로드해 보겠습니다. 랭체인의 [비정형 HTML 로더](https://python.langchain.com/docs/modules/data_connection/document_loaders/html)를 사용할 것입니다. 그러면 원시 HTML 파일이 파싱되어 LLM에 제공할 수 있는 형식으로 저장됩니다.마지막으로 위에서 정의한 스플리터 구성에 따라 각 문서를 분할합니다.

In [ ]:
import nltk
import os

# NLTK 데이터 디렉토리 설정
nltk_data_dir = '/home/ec2-user/nltk_data'
if not os.path.exists(nltk_data_dir):
    os.makedirs(nltk_data_dir)

nltk.data.path.append(nltk_data_dir)

# 리소스 다운로드
nltk.download('all', download_dir=nltk_data_dir)

In [ ]:
from langchain.document_loaders import UnstructuredHTMLLoader

# Initialize
html_doc_data_list = []
html_doc_link_list = []

# Loop through the downloaded HTML files in the directory
for html_link in html_link_list:
    file_name = html_link.split("/")[-1]
    file_path = '{}/{}'.format(html_files_dir, file_name)
    
    # Load the file content
    loader = UnstructuredHTMLLoader(file_path)
    data = loader.load()
    
    # Remove irrelevant text
    html_doc = data[0].page_content.replace("""Did this page help you? - Yes

Thanks for letting us know we're doing a good job!

If you've got a moment, please tell us what we did right so we can do more of it.

Did this page help you? - No

Thanks for letting us know this page needs work. We're sorry we let you down.

If you've got a moment, please tell us how we can make the documentation better.""", "").replace("""Javascript is disabled or is unavailable in your browser.

To use the Amazon Web Services Documentation, Javascript must be enabled. Please refer to your browser's Help pages for instructions.""", "")
    
    # Split our document into chunks
    texts = text_splitter.create_documents([html_doc])
    
    # Create a list of document chunks as well as a list of links
    for text in texts:
        html_doc_data_list.append(text.page_content)
        html_doc_link_list.append(html_link)

logging.info("Created {} chunks from the {} downloaded HTML files.".format(len(html_doc_data_list), len(html_link_list)))        

# Print the first chunk
#logging.info(html_doc_data_list[0])

#### c. 로드할 PDF 파일 준비<a id='Prepare%20PDF%20files%20for%20loading'></a>

다음 셀을 실행하여 일부 제너레이티브 AI 관련 백서를 채우면 채팅 어시스턴트가 관련 질문에 사실적으로 정확한 정보로 답변할 수 있습니다.

이 백서는 PDF 파일로 제공됩니다. 첫 단계로 이러한 파일을 `/data/rag/gen-ai-whitepapers/`라는 로컬 디렉터리에 다운로드합니다. 존재하지 않는 경우 필요한 디렉토리 구조가 생성됩니다.

In [ ]:
# A simple list of Gen AI whitepapers to index
pdf_link_list = [
    "https://www-cdn.anthropic.com/bd2a28d2535bfb0494cc8e2a3bf135d2e7523226/Model-Card-Claude-2.pdf"
]

# Set the sub-directory to store these PDF files
pdf_files_dir = "{}/{}".format(rag_dir, "gen-ai-whitepapers")
# Create the directory if it doesn't exist
os.makedirs(pdf_files_dir, exist_ok = True)

# Download and store each PDF file; overwrite existing file
for pdf_link in pdf_link_list:
    pdf_content = requests.get(pdf_link).content
    file_name = pdf_link.split("/")[-1]
    with open('{}/{}'.format(pdf_files_dir, file_name), "wb") as f:
        f.write(pdf_content)
        logging.info("Downloaded file '{}' to '{}'".format(file_name, pdf_files_dir))

logging.info("A total of {} PDF files were downloaded to '{}'.".format(len(pdf_link_list), pdf_files_dir))

이제 PDF 문서를 모두 다운로드했습니다. 이제 처리해 보겠습니다. [PyPDF2](https://pypdf2.readthedocs.io/en/3.0.0/) 라이브러리를 사용하여 텍스트를 추출해 보겠습니다. 마지막으로 위에서 정의한 스플리터 구성에 따라 각 문서를 분할합니다.

In [ ]:
from PyPDF2 import PdfReader

# Initialize
pdf_doc_data_list = []
pdf_doc_link_list = []

# Loop through the downloaded PDF files in the directory
for pdf_link in pdf_link_list:
    pdf_doc = ''
    file_name = pdf_link.split("/")[-1]
    file_path = '{}/{}'.format(pdf_files_dir, file_name)
    
    # Load the file content
    reader = PdfReader(file_path)
    
    # Loop through the pages
    for page in reader.pages:
        pdf_doc = pdf_doc + page.extract_text()
    
    # Split our document into chunks
    texts = text_splitter.create_documents([pdf_doc])
    
    # Create a list of document chunks as well as a list of links
    for text in texts:
        pdf_doc_data_list.append(text.page_content)
        pdf_doc_link_list.append(pdf_link)

logging.info("Created {} chunks from the {} downloaded PDF files.".format(len(pdf_doc_data_list), len(pdf_link_list)))        

# Print the first chunk
#logging.info(pdf_doc_data_list[0])

### C. 0b단계 및 0c단계: 임베딩 생성<a id='Step0band0c'></a>

이제 문서 청크가 준비되었으니, 이를 벡터화하여 임베딩을 만들어 보겠습니다. 이와 함께 AOSS 컬렉션의 색인에 삽입할 문서를 준비하겠습니다.

준비된 각 문서에는 다음 필드가 포함됩니다.
- `content` - 문서 청크의 실제 텍스트를 포함합니다.
- `콘텐츠 임베딩` - 해당 문서 청크의 임베딩을 포함합니다.
- `제목` - 콘텐츠가 인제스트된 위치의 이름에 대한 참조입니다.
- `소스` - 콘텐츠가 인제스트된 위치에 대한 참조

In [ ]:
# 문서 청크에서 임베딩을 생성하고 인덱싱할 문서를 준비합니다.
# 에서 사용할 수 있는 'prere_index_document_list' 도우미 함수를 사용합니다. /scripts/helper_functions.py

# 청크된 HTML 문서 처리하기
html_doc_list = prepare_index_document_list(br_embeddings, 'HTML', html_doc_data_list, html_doc_link_list)

# 청크된 PDF 문서 처리
pdf_doc_list = prepare_index_document_list(br_embeddings, 'PDF', pdf_doc_data_list, pdf_doc_link_list)

### D. 0d단계: 벡터 데이터베이스에 임베딩 저장<a id='Step0d'></a>

다음 셀을 실행하여 준비된 문서를 생성된 AOSS 컬렉션의 색인에 업로드합니다. 아래 함수는 병렬 처리 함수를 사용하여 문서를 색인에 업로드합니다. 병렬 워커 스레드의 수는 `max_workers` 변수에 의해 제어됩니다.

<div class="alert alert-block alert-info">
    <b>Note:</b> 아래 셀을 실행한 후 데이터를 읽을 수 있을 때까지 최대 30초가 걸릴 수 있습니다. boto3 오류가 있는 경우 이전 단계의 설정에 따라 API 호출이 자동으로 재시도됩니다.
</div>

<div class="alert alert-block alert-warning">  
    <b>Note:</b> 이 노트북을 작성할 당시 AOSS는 <i>벡터 검색</i> 컬렉션 유형에 대해 <i>id</i>를 사용하지 않았습니다. 따라서 다음 셀을 두 번 이상 실행하면 AOSS 색인에 중복 문서가 생성됩니다. 이 노트북을 실행하는 용도로는 괜찮습니다.
</div>

In [ ]:
# AOSS 인덱스로 가져올 함수를 정의합니다.
def os_import(article):
    """
    This function imports the documents and their metadata into the AOSS index.
    """
    aoss_py_client.index(index = index_name,
                         body={
                                "content-embedding": article['content-embedding'],
                                "content": article['content'],
                                "title": article['title'],
                                "source": article['source'],
                              }
                        )
    
# AOSS 컬렉션의 인덱스를 HTML 데이터로 병렬화하고 채웁니다.
process_map(os_import, html_doc_list, max_workers = max_workers)

# AOSS 컬렉션의 색인을 PDF 데이터로 병렬화하고 채웁니다.
process_map(os_import, pdf_doc_list, max_workers = max_workers) 

### E. 1~6단계: 채팅 단계 작성<a id='Step1to6'></a>

[검색 증강 생성 (RAG)](https://docs.aws.amazon.com/sagemaker/latest/dg/jumpstart-foundation-models-customize-rag.html) 아키텍처의 유용성을 설명하기 위해 벡터 데이터베이스를 검색하지 않고 LLM을 직접 호출해 보겠습니다. 여기서는 Amazon Bedrock 에이전트 또는 Anthropic Claude 모델의 성능에 대해 질문해 보겠습니다. LLM을 교육받았을 당시에는 이러한 기능을 사용할 수 없었기 때문에 LLM이 알지 못할 것으로 알고 있습니다.

In [ ]:
# Set the query
query = "What are Agents in Amazon Bedrock?"
#query = "How did Claude 2 do on the Graduate Record Exam?"
query_char_count, query_word_count = get_counts_from_text(query)
# Invoke the LLM
output = llm.generate([query])
# Read the response
query_response = output.generations[0][0].text
query_resp_char_count, query_resp_word_count = get_counts_from_text(query_response)

# Print the details
logging.info("\n\nHuman:\n{}\n\nAssistant:\n{}\n".format(query, query_response))

# Print the stats
logging.info("Query (prompt) stats: Character count = {}; Word count = {}"
             .format(query_char_count, query_word_count))
logging.info("Response stats: Character count = {}; Word count = {}"
             .format(query_resp_char_count, query_resp_word_count))

이 응답은 그럴듯해 보이지만 실제로는 올바르지 않습니다. 일반적으로 LLM은 질문에 답하려고 노력하지만 이 경우에는 환각 현상이 나타납니다. 이는 LLM이 정답을 제시하지 못한 사례입니다. RAG 아키텍처가 해결책을 제시할 수 있는 곳은 다음과 같습니다.

문서 색인에서 Amazon Bedrock 에이전트 또는 Anthropic Claude 모델의 성능에 대한 정보가 포함된 문서를 찾을 수 있는지 확인해 보겠습니다. 다음과 같은 두 가지 방법으로 이 작업을 수행할 수 있습니다.

방법 1: AOSS 파이썬 클라이언트의 `검색` 직접 사용하기.

In [ ]:
query = "What are Agents in Amazon Bedrock?"
#query = "How did Claude 2 do on the Graduate Record Exam?"
temp_embedding = br_embeddings.embed_query(text = query)
search_query = {"query": {"knn": {"content-embedding": {"vector": temp_embedding, "k": 5}}}}
results = aoss_py_client.search(index = index_name, body = search_query)
hits = results["hits"]["hits"]
logging.info("Found {} hit(s).".format(len(hits)))
for hit in hits:
    logging.info(hit["_source"]["source"])

방법 2: 랭체인의 'similarity_search'를 사용하여 AOSS 파이썬 클라이언트를 간접적으로 사용할 수 있습니다. 

In [ ]:
max_results = 5
query = "What are Agents in Amazon Bedrock?"
#query = "How did Claude 2 do on the Graduate Record Exam?"
docs = doc_search.similarity_search(
    # Our text query
    query = query,
    # The name of the field that contains our vector
    vector_field = "content-embedding",
    # The actual text field we are looking for
    text_field = "content",
    # The number of results we want to return
    k = max_results
)
logging.info("Specified {} max results. Found {} hit(s).".format(max_results, len(docs)))
for doc in docs:
    logging.info(docs[0].metadata['source'])

AOSS 컬렉션의 색인에 준비하고 저장한 문서에 Amazon Bedrock용 에이전트 또는 Falcon LLM용 RefinedWeb 데이터 세트에 대한 정보가 있는 것 같습니다. 이제 문서 색인에 올바른 정보가 있다는 것을 알았으니 [RetrievalQA 체인](https://python.langchain.com/docs/use_cases/question_answering/vector_db_qa) 을 설정해 보겠습니다. 이 체인을 통해 [프롬프트 템플릿](https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/), LLM 및 문서 색인을 제공하여 반환된 컨텍스트 문서를 기반으로 질문에 답변하는 질문 응답 체인을 구성할 수 있습니다.[stuff](https://python.langchain.com/docs/modules/chains/document/stuff) 체인 유형을 사용하겠습니다.

[프롬프트 템플릿](https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/) 은 언어 모델에 대한 프롬프트를 생성하기 위한 사전 정의된 레시피입니다. 아래에서 만든 변수에서는 컨텍스트 및 질문 입력 변수를 지정합니다. 이 변수는 RetrievalQA 체인이 쿼리 및 소스 문서로 채웁니다.

In [ ]:
# 프롬프트 템플릿 만들기
prompt_template = """\n\nHuman: 
다음 컨텍스트를 사용하여 끝에 있는 질문에 답하십시오. 
답을 모른다면 그냥 모른다고 말하고, 답을 만들려고 하지 마세요. 
유해한 콘텐츠는 넣지 마세요.
모든 답변은 한글로 작성해줘.

<context>
{context}
</context>

Question: {question}
\n\nAssistant:"""

PROMPT = PromptTemplate(
    template = prompt_template, input_variables = ["context", "question"]
)

# 검색 QA 체인 생성
qa = RetrievalQA.from_chain_type(llm = llm, 
                                 chain_type = "stuff", 
                                 retriever = doc_search.as_retriever(search_kwargs = {
                                     "vector_field": "content-embedding",
                                     "text_field": "content",
                                     "k": 5}),
                                 return_source_documents = True,
                                 chain_type_kwargs = {"prompt": PROMPT, "verbose": False},
                                 verbose = False)

# LLM에 질문하고 출처의 참고 자료와 함께 응답을 인쇄하십시오.
question = "What are Agents in Amazon Bedrock?"
#question = "How did Claude 2 do on the Graduate Record Exam?"
# LangChain RetrievalQA chain을 통해 임베딩 검색, LLM 등을 호출합니다.
response = qa(question, return_only_outputs = True)
# Parse the output
question, answer, context, title, source = parse_rqa_prompt_output(question, response)

# Print the details
prompt_template_with_result = prompt_template + answer + "\n"
logging.info(prompt_template_with_result.format(context=context, question=question))
logging.info("The context for the above question was retreived from here --> Title: {}; Source: {}"
             .format(title, source))

# Print the stats
query_char_count, query_word_count = get_counts_from_text(question)
logging.info("Question stats: Character count = {}; Word count = {}"
             .format(query_char_count, query_word_count))
query_resp_char_count, query_resp_word_count = get_counts_from_text(answer)
logging.info("Answer stats: Character count = {}; Word count = {}"
             .format(query_resp_char_count, query_resp_word_count))

이제 [대화형 리트리벌 체인](https://python.langchain.com/docs/expression_language/cookbook/retrieval#conversational-retrieval-chain)을 사용하여 한 단계 더 나아가겠습니다. 

LLM 자체로는 사용자가 마지막으로 입력한 내용이 기억나지 않습니다. 따라서 이전 대화 정보를 기억하고 LLM에 다시 제공할 수 있는 메커니즘이 필요합니다. 이 작업을 수행하려면 [ConversationBufferMemory](https://python.langchain.com/docs/modules/memory/adding_memory) 클래스와 함께 사용할 수 있습니다. 이렇게 하면 LLM과 대화를 진행하고 이전 대화를 메모리에 보관할 수 있습니다.

다음 셀은 검색 체인에 대화 요소를 추가하고 검색에 채팅 메모리를 추가할 수 있게 해줍니다. 이 체인은 문서 검색 전에 LLM 호출을 사용하여 대화 기록과 현재 질문을 하나의 새 질문으로 압축하여 문서 검색을 개선합니다.

먼저 프롬프트 템플릿을 만들고 체인을 인스턴스화하세요.

In [ ]:
# 대화형 검색 체인 만들기
cqa = ConversationalRetrievalChain.from_llm(llm = llm, 
                                            chain_type = "stuff", 
                                            condense_question_llm = llm,
                                            retriever = doc_search.as_retriever(search_kwargs = {
                                                "vector_field": "content-embedding",
                                                "text_field": "content",
                                                "k": 5}),
                                            return_source_documents = True,
                                            memory = ConversationBufferMemory(input_key = "question",
                                                                              output_key = "answer",
                                                                              memory_key = "chat_history",
                                                                              return_messages = True),
                                            verbose = False)
chat_history = []

첫 번째 질문을 합니다.

In [ ]:
question = "Claude 2는 대학원 기록 시험에서 어떻게 되었습니까?"
# 랭체인 대화형 검색 체인 체인을 통해 임베딩 검색, LLM 등을 호출합니다.
response = cqa.invoke({"question": question, "chat_history": chat_history})
# Parse and print the output
logging.info(convert_crc_chat_history_to_text(response))

두 번째 질문을 합니다. 이제 응답이 인쇄되면 이전 메시지 교환 내용이 메모리의 채팅 기록에 저장되어 그대로 인쇄됩니다.

In [ ]:
question = "Amazon Bedrock의 agent는 무엇인가요?"
# 랭체인 대화형 검색 체인 체인을 통해 임베딩 검색, LLM 등을 호출합니다.
response = cqa.invoke({"question": question, "chat_history": chat_history})
# Parse and print the output
logging.info(convert_crc_chat_history_to_text(response))

## 3. 어시스턴트와 채팅하기<a id='Chat%20with%20the%20assistant'></a>

이제 이 모든 것을 하나의 브라우저 인터페이스에 통합해 보겠습니다. 아래 호출을 통해 노트북 내부의 간단한 대화형 UI가 시작됩니다. 실행해서 질문을 시작하세요!

In [ ]:
# 'ChatUX' 클래스는 /scripts/helper_functions.py에 정의되어 있습니다.
# 인스턴스를 생성하고 어시스턴트와 대화형 채팅을 시작하세요

chatux = ChatUX(cqa)
chatux.start_chat()

## 4. Cleanup <a id='Cleanup'></a>

가장 좋은 방법은 더 이상 필요하지 않은 AWS 리소스를 삭제하는 것입니다. 이렇게 하면 불필요한 비용이 발생하지 않도록 할 수 있습니다.

<div class="alert alert-block alert-info">
<b>Note:</b> 기본적으로 모든 리소스는 세션이 끝날 때 정리됩니다. 세션 외부에서 이 노트북을 실행하는 경우 다음 셀을 실행하여 이 노트북을 통해 생성된 AOSS 리소스를 정리할 수 있습니다.
</div>

In [ ]:
# 헬퍼 함수 'delete_aoss_collection'(/scripts/helper_functions.py에서 제공하는)는 지정된 항목을 삭제합니다.
# AOSS 컬렉션과 그 안의 모든 인덱스.또한 지정된 데이터 액세스 정책, 암호화 정책 및 네트워크 정책도 삭제합니다.
if aoss_collection_created:
    delete_aoss_collection(aoss_client, collection_id, data_access_policy_name,
                           encryption_policy_name, network_policy_name)
else:
    logging.info("Skipping AOSS collection deletion.")

## 5. Conclusion <a id='Conclusion'></a>

이제 Amazon Bedrock에서 호스팅되는 대규모 언어 모델(LLM)을 사용하여 채팅 어시스턴트를 구축하는 방법을 살펴보았습니다. 이 과정에서 RAG(검색 증강 생성) 메커니즘이 환각을 방지하는데 어떻게 도움이 되는지도 시연했습니다. RAG를 사용하면서 Amazon Bedrock에 호스팅된 임베딩 모델을 사용하여 원시 텍스트를 벡터로 변환하는 방법과 Amazon OpenSearch Serverless 컬렉션에서 원시 텍스트를 저장하고 검색하는 방법을 보여 드렸습니다.

## 6. Frequently Asked Questions (FAQs) <a id='FAQs'></a>

**Q: 이 노트북에는 어떤 AWS 서비스가 사용됩니까?**

노트북을 실행하는 데 사용하는 항목에 따라 Amazon Bedrock, Amazon OpenSearch 서버리스, AWS ID 및 액세스 관리 (IAM), Amazon CloudWatch 및 Amazon SageMaker 노트북 인스턴스 (또는) Amazon SageMaker 스튜디오 노트북 인스턴스가 생성됩니다.

**Q: 오픈서치, 아마존 오픈서치 서버리스, 아마존 오픈서치 서비스의 차이점은 무엇입니까?**

OpenSearch는 로그 분석, 실시간 애플리케이션 모니터링, 클릭스트림 분석과 같은 사용 사례를 위한 완전한 오픈 소스 검색 및 분석 엔진입니다. 자세한 내용은 [오픈서치 문서](https://opensearch.org/docs/latest/)를 참조하십시오.

아마존 오픈서치 서비스는 오픈서치 클러스터를 위한 모든 리소스를 프로비저닝하고 실행합니다. 또한 장애가 발생한 OpenSearch Service 노드를 자동으로 탐지하고 교체하여 자체 관리형 인프라와 관련된 오버헤드를 줄입니다. 단일 API 호출 또는 콘솔에서 몇 번의 클릭으로 클러스터를 확장할 수 있습니다.

아마존 오픈서치 서버리스는 아마존 오픈서치 서비스를 위한 온디맨드 서버리스 구성입니다. 서버리스는 OpenSearch 클러스터를 프로비저닝, 구성 및 조정하는 데 따르는 운영상의 복잡성을 제거합니다. OpenSearch 클러스터를 자체 관리하고 싶지 않은 조직이나 대규모 클러스터를 운영할 전담 리소스나 전문 지식이 없는 조직에 적합한 옵션입니다. OpenSearch Serverless를 사용하면 기본 인프라 및 데이터 관리에 대해 걱정할 필요 없이 대량의 데이터를 쉽게 검색하고 분석할 수 있습니다.

**Q: Amazon OpenSearch 서버리스는 용량을 어떻게 관리합니까?**

Amazon OpenSearch 서버리스를 사용하면 용량을 직접 관리할 필요가 없습니다. OpenSearch 서버리스는 현재 워크로드를 기반으로 계정의 컴퓨팅 파워를 자동으로 확장합니다. 서버리스 컴퓨팅 파워는 오픈서치 컴퓨팅 유닛(OCU) 단위로 측정됩니다. 각 OCU는 6GiB 메모리와 해당 가상 CPU(vCPU), 그리고 Amazon S3로의 데이터 전송의 조합입니다. 오픈서치 서버리스의 분리된 아키텍처에 대한 자세한 내용은 [작동 방식](https://docs.aws.amazon.com/opensearch-service/latest/developerguide/serverless-overview.html#serverless-process)을 참조하십시오.

**Q: 아마존 베드록이 내 데이터를 캡처하고 저장합니까?**

Amazon Bedrock은 AWS 모델을 학습하거나 타사에 배포하는 데 사용자의 프롬프트 및 데이터를 사용하지 않습니다. 학습 데이터는 기본 Amazon Titan 모델을 교육하는 데 사용되거나 제3자에게 배포되지 않습니다. 사용 타임스탬프, 기록된 계정 ID, 서비스에서 기록한 기타 정보 등 기타 사용 데이터도 모델 학습에 사용되지 않습니다.

Amazon Bedrock은 Amazon Titan 모델을 미세 조정하는 용도로만 사용자가 제공한 미세 조정 데이터를 사용합니다. Amazon Bedrock은 기본 기반 모델 학습과 같은 다른 용도로는 미세 조정 데이터를 사용하지 않습니다.

각 모델 공급자는 모델을 업로드할 수 있는 에스크로 계정을 가지고 있습니다. Amazon Bedrock 추론 계정에는 이러한 모델을 호출할 권한이 있지만, 에스크로 계정 자체에는 Amazon Bedrock 계정에 대한 아웃바운드 권한이 없습니다. 또한 모델 제공자는 Amazon Bedrock 로그에 액세스하거나 고객 프롬프트 및 지속에 액세스할 수 없습니다.

Amazon Bedrock은 서비스 로그에 데이터를 저장하거나 기록하지 않습니다.

**Q: Amazon Bedrock은 어떤 모델을 지원합니까?**

[여기](https://docs.aws.amazon.com/bedrock/latest/userguide/what-is-bedrock.html#models-supported)로 가세요.

**Q: Amazon Bedrock의 온디맨드 처리량과 프로비저닝된 처리량의 차이는 무엇입니까?**

온디맨드 모드에서는 기간 약정 없이 사용한 만큼만 비용을 지불하면 됩니다. 텍스트 생성 모델의 경우 처리된 모든 입력 토큰과 생성된 모든 출력 토큰에 대해 요금이 부과됩니다. 임베딩 모델의 경우 처리된 모든 입력 토큰에 대해 요금이 부과됩니다. 토큰은 몇 개의 문자로 구성되며, 모델이 사용자 입력을 이해하고 결과를 생성하도록 유도하는 데 학습하는 기본 단위를 말합니다. 이미지 생성 모델의 경우 생성된 모든 이미지에 대해 요금이 부과됩니다.

Provisioned Throughput 모드에서는 특정 기본 모델이나 사용자 지정 모델의 모델 단위를 구매할 수 있습니다. 프로비저닝된 처리량 모드는 주로 보장된 처리량이 필요한 대규모의 일관된 추론 워크로드를 위해 설계되었습니다. 사용자 지정 모델은 프로비저닝된 처리량을 사용해야만 액세스할 수 있습니다. 모델 단위는 분당 처리되는 입력 또는 출력 토큰의 최대 수로 측정되는 특정 처리량을 제공합니다. 시간당 요금이 부과되는 이 프로비저닝된 처리량 요금제를 사용하면 1개월 또는 6개월 약정 기간 중에서 유연하게 선택할 수 있습니다.

**Q: Amazon Bedrock에 대한 고객 레퍼런스는 어디에서 찾을 수 있습니까?**

[여기](https://aws.amazon.com/bedrock/testimonials/)로 가세요.

**Q: 신속한 엔지니어링을 위한 리소스는 어디서 찾을 수 있나요?**

[프롬프트 엔지니어링 가이드](https://www.promptingguide.ai/).

**Q: 아마존 베드록을 사용하려면 랭체인이 필수인가요?**

아니요. [베드록 API](https://docs.aws.amazon.com/bedrock/latest/APIReference/welcome.html) 또는 언어별 [AWS SDK](https://aws.amazon.com/developer/tools/)를 사용하여 아마존 베드록과 상호 작용할 수 있습니다. LangChain을 사용하면 이 아키텍처와 관련된 다양한 구성 요소 간의 상호 작용과 관련된 단계를 간단하게 제어할 수 있습니다. 

**Q: LangChain을 시작하려면 어떻게 해야 하나요?**

[여기](https://python.langchain.com/docs/get_started/introduction)로 가세요.

**Q: 이 노트북에서 사용되는 AWS 서비스의 요금 정보는 어디에서 찾을 수 있습니까?**

- 아마존 베드락 가격 책정 - [여기](https://aws.amazon.com/bedrock/pricing/)로 이동하세요.
<i>- 아마존 오픈서치 서버리스 가격 책정 - [여기](https://aws.amazon.com/opensearch-service/pricing/)로 이동하여 서버리스 섹션으로 이동하십시오.</i>
— AWS ID 및 액세스 관리 (IAM) 요금 — 무료.
- 아마존 클라우드워치 요금 - [여기](https://aws.amazon.com/cloudwatch/pricing/)로 이동하십시오.
- 아마존 세이지메이커 노트북 인스턴스 (또는) 아마존 세이지메이커 스튜디오 노트북 요금 - [여기](https://aws.amazon.com/sagemaker/pricing/)로 이동하십시오.